In [2]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, ConvLSTM2D, Conv3D, BatchNormalization, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
import data_preprocessing as dpp

tf.random.set_seed(1)
np.random.seed(1)

# Data Preprocessing

In [3]:
img_folder = 'data/data1-(8.7μm)'

img_tensors = dpp.preprocess_images(img_folder, end_i=40)

print(img_tensors)

[[[[0.91372549 0.91372549 0.91372549 1.        ]
   [0.90980392 0.90980392 0.90980392 1.        ]
   [0.90196078 0.90196078 0.90196078 1.        ]
   ...
   [0.61176471 0.61176471 0.61176471 1.        ]
   [1.         1.         0.         1.        ]
   [0.41176471 0.41176471 0.41176471 1.        ]]

  [[0.88627451 0.88627451 0.88627451 1.        ]
   [0.90980392 0.90980392 0.90980392 1.        ]
   [0.90196078 0.90196078 0.90196078 1.        ]
   ...
   [0.61176471 0.61176471 0.61176471 1.        ]
   [1.         1.         0.         1.        ]
   [0.41176471 0.41176471 0.41176471 1.        ]]

  [[0.88627451 0.88627451 0.88627451 1.        ]
   [0.88627451 0.88627451 0.88627451 1.        ]
   [0.89019608 0.89019608 0.89019608 1.        ]
   ...
   [0.48235294 0.48235294 0.48235294 1.        ]
   [1.         1.         0.         1.        ]
   [0.38823529 0.38823529 0.38823529 1.        ]]

  ...

  [[0.30196078 0.30196078 0.30196078 1.        ]
   [0.8745098  0.8745098  0.0196078

In [4]:
inputs = np.load("data/data_/data_2410_input.npy")
outputs = np.load("data/data_/data_2410_output.npy")

In [5]:
inputs.shape, outputs.shape

((47, 14, 850, 850, 4), (47, 1, 850, 850, 4))

# Modeling

In [6]:
# 자 모델 구축 들갑니다~
model = Sequential()
model.add(ConvLSTM2D(filters=40, 
                     kernel_size=(3,3), 
                     input_shape=(None,850,850,4), 
                     padding="same", 
                     return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=40, 
                     kernel_size=(3,3), 
                     padding="same", 
                     return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=40, 
                     kernel_size=(3,3), 
                     padding="same", 
                     return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=40, 
                     kernel_size=(3,3), 
                     padding="same", 
                     return_sequences=False))
model.add(BatchNormalization())
model.add(Lambda(lambda x: tf.expand_dims(x, axis=1)))
model.add(Conv3D(filters=4, 
                 kernel_size=(3,3,3), 
                 activation="sigmoid",
                 padding="same"))
model.compile(loss='binary_crossentropy',optimizer='adadelta', metrics = ['accuracy'])
callbacks_val_acc = [EarlyStopping(monitor='val_accuracy', patience=3, mode='max', restore_best_weights=True),
                     ModelCheckpoint("ClimateAI.keras", monitor='val_accuracy', save_best_only=True)]
callbacks_val_loss = [EarlyStopping(monitor='val_loss', patience=3, mode='min', restore_best_weights=True),
                      ModelCheckpoint("ClimateAI.keras", monitor='val_loss', save_best_only=True)]
callbacks = [callbacks_val_acc, callbacks_val_loss]

model.summary()

/Volumes/looparam/#1 Coding/#10 Python/AcademicFestival/AF/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d (ConvLSTM2D)        │ (None, None, 850, 850, │        63,520 │
│                                 │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, None, 850, 850, │           160 │
│ (BatchNormalization)            │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_1 (ConvLSTM2D)      │ (None, None, 850, 850, │       115,360 │
│                                 │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, None, 850, 850, │           160 │
│ (BatchNormalization)            │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_2 (ConvLSTM2D)      │ (None, None, 850, 850, │       115,360 │
│                                 │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, None, 850, 850, │           160 │
│ (BatchNormalization)            │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_3 (ConvLSTM2D)      │ (None, 850, 850, 40)   │       115,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 850, 850, 40)   │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 1, 850, 850,    │             0 │
│                                 │ 40)                    │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d (Conv3D)                 │ (None, 1, 850, 850, 4) │         4,324 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 414,564 (1.58 MB)

 Trainable params: 414,244 (1.58 MB)

 Non-trainable params: 320 (1.25 KB)

# Train

In [ ]:
model.fit(inputs, outputs, epochs=1, batch_size=5, validation_split=0.1, callbacks=callbacks)